# Smart Parking System Producer

**Se realiza primeramente un código que realice datos simulados para poder enviar datos a través de Apache Kafka mediante un topico en especifico**

## 1.- Se realiza la importación de las principales librerías a usar

1. Kafka - Para realizar la conexión a Apache Kafka y enviar data al topico correspondiente
2. Random - Generación de data aleatoria según distintas listas
3. Json - Para envío de un json mediante el producer 
4. Datetime - Obtener fechas y distintos tipos de transformaciones posibles
5. time - Necesario para realizar sleeps de 1 segundo por cada envío 

In [1]:
import random
import json
import uuid
import time
import os
import requests
import pyowm
from kafka import KafkaProducer
from datetime import datetime, timedelta

## 2.- Primeros Valores a necesitar simular

#### El Json en la entrada del sistema tiene la siguiente característica:

{
  "parking_name":"Parking Tres Cruces",
  "parking_address":"Bvr. Gral. Artigas 1825, Montevideo, Uruguay",
  "description":"Parking que se encuentra en el shopping de tres cruces",
  "timestamp":"2022-05-03 11:12:51",
  "device_address":"0013A2004147A0E5",
  "device_latitude":-34.89429684062983,
  "device_longitude":-56.165979541252796,
  "temperature":17.01,
  "humidity":67,
  "parking_uuid":"cf0158d3-42e1-4744-b477-cf45b7de960a",
  "parking_id":"1",
  "level_id":"1",
  "area_id":"1",
  "area_name":"A",
  "spots":"5",
  "slots":[
    {
      "slot_id":"1",
      "state":true
    },
    {
      "slot_id":"2",
      "state":false
    },
    {
      "slot_id":"3",
      "state":true
    },
    {
      "slot_id":"4",
      "state":true
    },
    {
      "slot_id":"5",
      "state":true
    }
  ]
}

#### Se realiza carga de principales variables para el Simulador de Productor:

In [2]:
# Se establecen los rangos a establecer con random

dirgeneral = "/opt/smart-parking/Python/Generator/Parkings/"
directorios = os.listdir(dirgeneral)

print("Indique en número sobre que Parking desea realizar la simulación: \n")
for i in range(1, len(directorios)+1):
    print(str(i)+".- "+directorios[i-1])
num = input("Ingrese su opción (1): ")
parking = directorios[0] if num == "" else directorios[int(num) - 1]

with open(dirgeneral+parking+'/producerTresXShoppingv2.json') as f:
    data = json.load(f)

with open(dirgeneral+parking+'/'+parking+'Devices.txt', 'r') as f:
    devices = f.readlines()

print("\nSeleccione alguno de los siguientes dispositivos virtuales: ")
for i in range(1, len(devices)+1):
    line = devices[i-1].replace("\n","")
    print(str(i)+".- "+line)
num2 = input("Ingrese su opción (1): ")
num2 = 1 if num2 == "" else int(num2)

device = devices[num2-1].replace("\n", "")
data['device_address'] = device
devices.remove(devices[num2-1])
devices

with open(dirgeneral+parking+'/ActiveDevices/'+device+'.txt', 'w') as f:
    f.write("1\n")
    
try:
    with open(dirgeneral+parking+'/AllActiveDevices.txt', 'a+') as f:
        f.write(device+"\n")
except:
    with open(dirgeneral+parking+'/AllActiveDevices.txt', 'w+') as f:
        f.write(device)
        
with open(dirgeneral+parking+'/'+parking+'Devices.txt', 'w') as f:
    f.writelines(devices)

num_puestos = input("\nIndique cuantos puestos desea simular (10): ")
num_puestos = 10 if num_puestos == "" else int(num_puestos)
data['spots'] = str(num_puestos)

# Format the slots object data
for i in range(1,num_puestos + 1):
    data['slots'].append({})
    
nivel = input("Indique el Nivel del estacionamiento (1): ")
nivel = "1" if nivel == "" else nivel
data['level_id'] = nivel 
    
area = input("Indique el numero de sector del estacionamiento (1): ")
area = "1" if area == "" else area
data['area_id'] = area 

areaName = input("Indique el nombre de sector del estacionamiento (A): ")
areaName = "A" if areaName == "" else areaName  
data['area_name'] = areaName

# Values used to random function
lista_estado_puestos = [False for n in range(1,num_puestos + 1)]
lista_auxTemp = [0 for n in range(1, num_puestos + 1)]
#rango = [2, 5, 10, 20, 30, 45, 60, 120, 240]
rango = [1, 2]
slot_states = [True, False]

# Rangos de fechas para horario nocturno, en el cual los carros se encuentran dentro del Parking
Turno_inicio = str(datetime.now().day) +'/'+str(datetime.now().month) +'/'+str(datetime.now().year) +' 22:00:00'
Turno_final = str(datetime.now().day + 1) +'/'+str(datetime.now().month) +'/'+str(datetime.now().year) +' 08:00:00'

# Obtener fecha de primera consulta para el tiempo
APIKEY='055bcf46839359ea922b925c3d7cd860'        # your API Key here as string
owm = pyowm.OWM(APIKEY)                       # Use API key to get data
mgr = owm.weather_manager()
weather = mgr.weather_at_place('Montevideo,UY').weather
getTimeWeather = datetime.now() + timedelta(minutes=30)
data['temperature'] = weather.temperature(unit='celsius')['temp']
data['humidity'] = weather.humidity
data['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Initialize a Kafka Producer
producer = KafkaProducer(bootstrap_servers=['hadoop-namenode:9092'], api_version=(0,10,0), value_serializer=lambda x: json.dumps(x).encode('utf-8'))
topic = data['parking_id']+"_"+data['level_id']+"_"+data['area_id']+"_"+data['area_name']
print("El Topico a consumir es - "+topic)

Indique en número sobre que Parking desea realizar la simulación: 

1.- PuntaCarretasShopping
2.- TresCrucesShopping
Ingrese su opción (1): 2

Seleccione alguno de los siguientes dispositivos virtuales: 
1.- 50B44DD728621400
2.- 15DF507355D03434
3.- 257AB93BB423C258
4.- 22D58E8B09560713
5.- 5F4AB3EBEE3ACF7F
6.- 9C099CAF98176CC2
7.- 00CB3A037743E2E4
8.- 22B8D48D50DD67E3
9.- 2271B45C4F2F019F
Ingrese su opción (1): 

Indique cuantos puestos desea simular (10): 
Indique el Nivel del estacionamiento (1): 
Indique el numero de sector del estacionamiento (1): 
Indique el nombre de sector del estacionamiento (A): 
El Topico a consumir es - 1_1_1_A


## Loop para el envío de datos continuo con ciertas modificaciones en su estructura

In [3]:
while True:
    now = datetime.now()
    
    with open(dirgeneral+parking+'/ActiveDevices/'+device+'.txt', 'r') as f:
        status = f.readline()
    if status == '0\n':
        break
        
    if now >= datetime.strptime(Turno_inicio, '%d/%m/%Y %H:%M:%S') and now <= datetime.strptime(Turno_final, '%d/%m/%Y %H:%M:%S'):
        #auxTemp1 = auxTemp2 = auxTemp3 = auxTemp4 = auxTemp5 = 0
        #status_slot1 = status_slot2 = status_slot3 = status_slot4 = status_slot5 = False
        lista_estado_puestos = [False for n in range(1,num_puestos + 1)]
        lista_auxTemp = [0 for n in range(1, num_puestos + 1)]
    else:
        
        Turno_inicio = str(now.day) +'/'+str(now.month) +'/'+str(now.year) +' 22:00:00'
        Turno_final = str(now.day + 1) +'/'+str(now.month) +'/'+str(now.year) +' 08:00:00'
        
        for i in range(0,len(lista_estado_puestos)):
            if lista_auxTemp[i] == 0:
                minutes = random.choice(rango)
                lista_auxTemp[i] = now + timedelta(minutes=minutes)
                lista_estado_puestos[i] = random.choice(slot_states)
            elif now > lista_auxTemp[i]:
                minutes = random.choice(rango)
                lista_auxTemp[i] = now + timedelta(minutes=minutes)
                lista_estado_puestos[i] = random.choice(slot_states)
        
        if getTimeWeather.hour == now.hour and getTimeWeather.minute == now.minute:
            weather = mgr.weather_at_place('Montevideo,UY').weather
            data['temperature'] = weather.temperature(unit='celsius')['temp']
            data['humidity'] = weather.humidity
            getTimeWeather = now + timedelta(minutes=30)
        else:
            data['temperature'] = weather.temperature(unit='celsius')['temp']
            data['humidity'] = weather.humidity

    data['timestamp'] = now.strftime('%Y-%m-%d %H:%M:%S')
     
    for i in range(0,len(lista_estado_puestos)):
        data['slots'][i]['slot_id'] = str(i+1)
        data['slots'][i]['state'] = lista_estado_puestos[i]
    
    print(now.strftime('%Y-%m-%d %H:%M:%S')+" - "+str(data) + "\n")
    
    producer.send(topic, value=data)
    
    time.sleep(1)

2022-05-04 23:56:14 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:56:14', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:56:15 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:56:24 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:56:24', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:56:25 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:56:34 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:56:34', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:56:35 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:56:44 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:56:44', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:56:45 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:56:55 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:56:55', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:56:56 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

...

2022-05-04 23:57:15 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:57:15', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:57:16 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:57:25 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:57:25', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:57:26 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:57:35 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:57:35', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:57:36 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:57:45 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:57:45', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:57:46 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

2022-05-04 23:57:55 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montevideo, Uruguay', 'description': 'Parking que se encuentra en el shopping de tres cruces', 'timestamp': '2022-05-04 23:57:55', 'device_address': '50B44DD728621400', 'device_latitude': -34.89429684062983, 'device_longitude': -56.165979541252796, 'temperature': 9.84, 'humidity': 93, 'parking_uuid': 'cf0158d3-42e1-4744-b477-cf45b7de960a', 'parking_id': '1', 'level_id': '1', 'area_id': '1', 'area_name': 'A', 'spots': '10', 'slots': [{'slot_id': '1', 'state': False}, {'slot_id': '2', 'state': False}, {'slot_id': '3', 'state': False}, {'slot_id': '4', 'state': False}, {'slot_id': '5', 'state': False}, {'slot_id': '6', 'state': False}, {'slot_id': '7', 'state': False}, {'slot_id': '8', 'state': False}, {'slot_id': '9', 'state': False}, {'slot_id': '10', 'state': False}]}

2022-05-04 23:57:56 - {'parking_name': 'Parking Tres Cruces', 'parking_address': 'Bvr. Gral. Artigas 1825, Montev

In [4]:
# Rutina de Devolución y Limpieza
activesDir = os.listdir(dirgeneral+parking+"/ActiveDevices")
#print(devices)
if device+".txt" in activesDir:
    devices.append(device+"\n")
with open(dirgeneral+parking+'/'+parking+'Devices.txt', 'w') as f:
    f.writelines(devices)
#print(devices)    
os.remove(dirgeneral+parking+"/ActiveDevices/"+device+".txt")
activesDir = os.listdir(dirgeneral+parking+"/ActiveDevices")
with open(dirgeneral+parking+'/AllActiveDevices.txt', 'w+') as f:
    dev2 = []
    for file in activesDir:
        dev2.append(file.replace(".txt",""))
    f.writelines(dev2)